# Load libraries

In [10]:
import pandas as pd
import numpy as np

In [292]:
from matplotlib import*
import matplotlib.pyplot as plt
from matplotlib.cm import register_cmap

import seaborn as sns

from bokeh.plotting import figure, output_file, show, ColumnDataSource
from bokeh.models import HoverTool, FactorRange
from bokeh.io import show, output_file
from bokeh.models.tickers import FixedTicker
from bokeh.models import Arrow, OpenHead, NormalHead, VeeHead

from pprint import pprint

In [12]:
from scipy.cluster.hierarchy import fcluster
from scipy.cluster.hierarchy import linkage, dendrogram
from scipy import stats


In [13]:
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
from sklearn.preprocessing import normalize
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA as sklearnPCA

# Functions

In [117]:
# For labeling purpose. Create a list of PCx or Principal Component x
def pc(num, lng):
    if lng == 0:
        prefix = 'PC'
    else:
        prefix = 'Principal Component '
    pc_list = []
    for i in np.arange(1, num+1):
        pric = prefix+str(i)
        pc_list.append(pric)
    return pc_list

In [177]:
# Expected Proportion of the kth Largest Piece
# Adobted from https://blogs.sas.com/content/iml/2017/08/02/retain-principal-components.html

def exprop(n):
    ex_list = []
    for i in np.arange(1,n+1):
        tempsum = 0
        for j in np.arange(i, n+1):
            tempsum = tempsum + (1/j)
        cor = 100*tempsum/n
        ex_list.append(cor)
    return ex_list

# Read data from spreadsheet

In [18]:
raw_input = pd.read_csv('data/cleaned.csv', index_col='County')

In [40]:
raw_input

,pct_rur_10,pop_10,pop_00,pop_90,pop_80,pop_70,hsg_10,hsg_00,hsg_90,hsg_80,...,cp_id_fm_02,cp_id_acr_02,cp_fl_fm_02,cp_fl_acr_02,cp_sm_fm_02,cp_sm_acr_02,wd_fm_02,wd_acr_02,wd_p1_fm_02,wd_p1_acr_02
County,,,,,,,,,,,,,,,,,,,,,
anderson,34.7,75129,71330,68250,67346,60300,34717,32451,29323,25849,...,72.0,2364.0,14.0,93.0,3.0,16.0,369.0,12974.0,192.0,3790.0
bedford,55.6,45058,37586,30411,27916,25039,18360,14990,12638,10814,...,191.0,5373.0,37.0,412.0,1.0,12.0,859.0,38296.0,610.0,22449.0
benton,78.5,16489,16537,14524,14901,12126,8975,8595,7107,6526,...,167.0,6895.0,13.0,66.0,2.0,6.0,346.0,19621.0,175.0,5589.0
bledsoe,100.0,12876,12367,9669,9478,7643,5718,5142,3771,3406,...,84.0,4771.0,11.0,189.0,6.0,6.0,318.0,21127.0,182.0,6505.0
blount,32.6,123010,105823,85969,77770,63744,55266,47059,36532,30836,...,143.0,2234.0,28.0,2231.0,2.0,50.0,712.0,17460.0,424.0,8212.0
bradley,33.0,98963,87965,73712,67547,50686,41395,36820,29562,24705,...,80.0,2229.0,13.0,156.0,6.0,30.0,473.0,23121.0,283.0,7998.0
campbell,55.0,40716,39854,35079,34923,26045,19966,18527,14817,13250,...,63.0,698.0,22.0,393.0,3.0,145.0,245.0,8297.0,160.0,4235.0
cannon,81.1,13801,12826,10467,10234,8467,6037,5420,4368,4002,...,87.0,2438.0,26.0,346.0,2.0,15.0,567.0,30387.0,387.0,17462.0
carroll,83.1,28522,29473,27514,28285,25741,13184,13055,11783,11306,...,330.0,14790.0,19.0,439.0,11.0,70.0,576.0,36829.0,288.0,8978.0


In [48]:
metrics = list(raw_input.columns)
counties = list(raw_input.index)
print(len(metrics), len(counties))

188 95


# Step for principal component analysis

In [22]:
# Create scaler: scaler
scaler = StandardScaler()

# Fit the pipeline to 'samples'
scaler_df = pd.DataFrame(scaler.fit_transform(raw_input))

In [52]:
scaler_df.columns = metrics
scaler_df.index = counties

scaler_df

In [25]:
# Covariance matrix
cov_df = pd.DataFrame(np.cov(scaler_df.T))

cov_df.index = metrics
cov_df.columns = metrics
cov_df

In [28]:
#Perform eigendecomposition on covariance matrix
eig_vals, eig_vecs = np.linalg.eig(cov_df)

eigvals_df=pd.DataFrame(eig_vals)
eigvals_df.columns = ['Eigenvalue']
eigvals_df.index = pc(eigvals_df.shape[0], 1)
eigvals_df.head(10)

In [123]:
eigvecs_df=pd.DataFrame(eig_vecs)
eigvecs_df.columns=pc(eig_vecs.shape[1],0)
eigvecs_df.index = metrics
eigvecs_df

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,...,PC179,PC180,PC181,PC182,PC183,PC184,PC185,PC186,PC187,PC188
pct_rur_10,(0.0265166408309+0j),(-0.138636203398+0j),(-0.0856888241795+0j),(-0.0356870640313+0j),(-0.0553088725031+0j),(-0.00430042599827+0j),(-0.104310037579+0j),(0.0376571953585+0j),(0.0554641863231+0j),(0.0326329762538+0j),...,(0.0426049622303+0.016379141892j),(0.0601287753815-0.0337779325833j),(0.0601287753815+0.0337779325833j),(0.000498699205324+0j),(0.0132835177368+0j),(0.0635756813007+0j),(-0.0314871553222-0.0234298000658j),(-0.0314871553222+0.0234298000658j),(-0.00446678127175-0.0145359304693j),(-0.00446678127175+0.0145359304693j)
pop_10,(-0.00459806255712+0j),(0.167579793098+0j),(0.0668379152236+0j),(0.0800206104921+0j),(-0.127333413807+0j),(0.0177761586216+0j),(0.014114854656+0j),(-0.0133934892393+0j),(0.0629233264485+0j),(0.0266703045673+0j),...,(-0.014703607679-0.0129205844372j),(-0.0011381397523+0.00350394656425j),(-0.0011381397523-0.00350394656425j),(-0.00952870268643+0j),(-0.00400125868397+0j),(-0.00772609307149+0j),(-0.000604077769393-0.00488640393544j),(-0.000604077769393+0.00488640393544j),(-0.00601932431904+0.000533141504357j),(-0.00601932431904-0.000533141504357j)
pop_00,(-0.0022092437395+0j),(0.161160022825+0j),(0.0659584823748+0j),(0.0972946608008+0j),(-0.138021176769+0j),(0.015522164782+0j),(0.02511309736+0j),(-0.020834002238+0j),(0.0747222463578+0j),(0.0262646745501+0j),...,(-0.0181040275855-0.00297217151087j),(-0.00886262338177+0.00973230397738j),(-0.00886262338177-0.00973230397738j),(-0.0265343789063+0j),(-0.00595936372212+0j),(-0.0230006695353+0j),(0.00484160888404-0.00494889599666j),(0.00484160888404+0.00494889599666j),(-0.0102486155986-0.0118819404711j),(-0.0102486155986+0.0118819404711j)
pop_90,(-1.85373259721e-05+0j),(0.155598765656+0j),(0.0653985597991+0j),(0.111391116349+0j),(-0.146074910722+0j),(0.0137218226156+0j),(0.0333701454886+0j),(-0.0256699083229+0j),(0.0790908049469+0j),(0.0227415770288+0j),...,(0.0213220182049-0.00866055325426j),(0.00369453217878+0.0248414307622j),(0.00369453217878-0.0248414307622j),(-0.00863473797543+0j),(-0.00494812108796+0j),(0.031218042656+0j),(0.0201468097025-0.00245200384814j),(0.0201468097025+0.00245200384814j),(-0.00361679623221-0.0139027022114j),(-0.00361679623221+0.0139027022114j)
pop_80,(0.00112528848132+0j),(0.1522138298+0j),(0.0651656648638+0j),(0.121690953681+0j),(-0.149232867835+0j),(0.0110309484796+0j),(0.0394087477118+0j),(-0.0294230987494+0j),(0.0790303932664+0j),(0.0192630406612+0j),...,(0.0240236200808-0.0146079056016j),(0.0404100668381-0.0162473120763j),(0.0404100668381+0.0162473120763j),(0.00546413986947+0j),(0.0139635490636+0j),(-0.0236291809333+0j),(-0.052188072334-0.0270536582019j),(-0.052188072334+0.0270536582019j),(-0.0310226944591-0.000892341175645j),(-0.0310226944591+0.000892341175645j)
pop_70,(0.00263997407593+0j),(0.147215690528+0j),(0.0656042780366+0j),(0.128064417876+0j),(-0.156636042924+0j),(0.0133186944491+0j),(0.0401968589562+0j),(-0.0266873709363+0j),(0.0801542917352+0j),(0.020364787074+0j),...,(0.0184157066891+0.038248971579j),(0.00886381020772-0.0381303859651j),(0.00886381020772+0.0381303859651j),(0.0161159685366+0j),(0.0117599004416+0j),(0.00522381405635+0j),(0.00716586048474-0.0290285210832j),(0.00716586048474+0.0290285210832j),(0.0502736257483-0.0263791273804j),(0.0502736257483+0.0263791273804j)
hsg_10,(-0.00358244743666+0j),(0.167393791181+0j),(0.0645722045125+0j),(0.0882736675552+0j),(-0.128530847393+0j),(0.0153710554859+0j),(0.0191785197494+0j),(-0.010073167987+0j),(0.054841640867+0j),(0.0235408948174+0j),...,(0.0274124019898+0.0141597053372j),(0.134935105381-0.0130031724661j),(0.134935105381+0.0130031724661j),(-0.0549121093989+0j),(-0.0200279685283+0j),(0.064785877548+0j),(-0.0870028374225-0.0604014292965j),(-0.0870028374225+0.0604014292965j),(-0.0664237317469-0.0315387466668j),(-0.0664237317469+0.0315387466668j)
hsg_00,(-0.00166976403791+0j),(0.162300174159+0j),(0.064102823536+0j),(0.103503704806+0j),(-0.137400016619+

In [95]:
# Create a PCA model with 30 components: pca
pca = sklearnPCA(n_components=30)

# Fit the PCA instance to the scaled samples
pca.fit(scaler_df)

# Transform the scaled samples: pca_features
pca_features = pca.transform(scaler_df)

In [275]:
pcaft_df = pd.DataFrame(pca_features)
pcaft_df.columns = pc(pcaft_df.shape[1],0)
pcaft_df.index = counties
pcaft_df.drop(list(pcaft_df.columns[np.arange(3,pcaft_df.shape[1])]), axis=1, inplace=True)

In [282]:
pcacom_df = pd.DataFrame(pca.components_.T)
pcacom_df.columns = pc(pcacom_df.shape[1],0)
pcacom_df.index = metrics
pcacom_df.drop(list(pcacom_df.columns[np.arange(3,pcacom_df.shape[1])]), axis=1, inplace=True)

In [283]:
pcacom_df

,PC1,PC2,PC3
pct_rur_10,-0.026517,-0.138636,-0.085689
pop_10,0.004598,0.167580,0.066838
pop_00,0.002209,0.161160,0.065958
pop_90,0.000019,0.155599,0.065399
pop_80,-0.001125,0.152214,0.065166
pop_70,-0.002640,0.147216,0.065604
hsg_10,0.003582,0.167394,0.064572
hsg_00,0.001670,0.162300,0.064103
hsg_90,-0.000610,0.156885,0.064390
hsg_80,-0.001218,0.152686,0.064710


In [276]:
pcaft_df

,PC1,PC2,PC3
anderson,-7.332806,2.383207,-1.462753
bedford,16.606917,-1.754641,-1.416658
benton,-4.443341,-2.766427,-0.575457
bledsoe,-2.663562,-2.707438,-3.624859
blount,3.629524,4.585776,-1.504190
bradley,0.006174,3.420939,-2.553215
campbell,-8.624536,1.662236,-2.081417
cannon,0.836037,-1.248077,-4.083825
carroll,3.616707,-6.814513,5.789249
carter,-7.197445,1.736370,-3.013973


In [270]:
# Scree plot values: determin how many component explain over 90% of total variation.

vari= eigvals_df=pd.DataFrame(100* pca.explained_variance_ratio_)
vari['1'] = ''
vari['2'] = ''
vari.iloc[0,1] = vari.iloc[0,0]
vari.iloc[0,2] = eig_vals[0].real
for i in np.arange(1, vari.shape[0]):
    vari.iloc[i,1] = vari.iloc[i,0] + vari.iloc[i-1,1]
    vari.iloc[i,2] = eig_vals[i].real

vari.columns = ['Exp. Variance (%)', 'Cumulative (%)', 'Eigenvalue']
vari.index = pc(vari.shape[0],0)
vari

,Exp. Variance (%),Cumulative (%),Eigenvalue
PC1,36.931857,36.9319,70.1705
PC2,13.080779,50.0126,24.8535
PC3,11.508304,61.5209,21.8658
PC4,4.377758,65.8987,8.31774
PC5,3.959178,69.8579,7.52244
PC6,3.342003,73.1999,6.34981
PC7,1.972549,75.1724,3.74784
PC8,1.527689,76.7001,2.90261
PC9,1.303871,78.004,2.47736
PC10,1.261902,79.2659,2.39761


In [272]:
# Scree Plot
# Tooltips to be fixed

output_file("scree.html")

hover = HoverTool(
    tooltips=[
    ("Eigenvalue", "@egval_y"),
    ], mode = "vline"
)

source = ColumnDataSource(data=dict(
exvar_y = vari['Exp. Variance (%)'],
acvar_y = vari['Cumulative (%)'],
egval_y = vari['Eigenvalue'],
   
))



p = figure(title="Variance Explained / Eigenvalues",
           x_range = FactorRange(factors=pc(vari.shape[0],0)),
           y_range = (0,100),
           plot_width=1300, plot_height=700,
           x_axis_label='Principal Components',
           tools=[hover])

pc_x = np.arange(1, vari.shape[0]+1)
exvar_y = vari['Exp. Variance (%)']
acvar_y = vari['Cumulative (%)']
egval_y = vari['Eigenvalue']
expro_y = exprop(vari.shape[0])

#p.background_fill_color = "beige"
p.xaxis.ticker = pc_x
p.yaxis.ticker = FixedTicker(ticks=np.arange(0,101,10))
p.ygrid.ticker = FixedTicker(ticks=np.arange(0,101,10))
p.xgrid.grid_line_color = None
# p.xgrid.grid_line_dash = "dotted"
p.ygrid.grid_line_color = 'silver'
p.ygrid.grid_line_dash = "dotted"
p.xaxis.major_tick_line_color = None

p.vbar(x=pc_x, width=0.9, bottom=0, top=egval_y, color="lightblue", legend="Eigenvalue")

p.line(pc_x, exvar_y, legend="Exp. Variance (%)", line_color="red", line_dash="solid", line_width = 2)
p.circle(pc_x, exvar_y, legend="Exp. Variance (%)", line_color="red", size=9, fill_color='white')

p.line(pc_x, acvar_y, legend="Cumulative (%)", line_color="green", line_dash="solid", line_width = 2)
p.circle(pc_x, acvar_y, legend="Cumulative (%)", line_color="green", size=9, fill_color="white")



p.legend.location = "center_right"

p.line(pc_x, expro_y, legend="Broken-Stick Model", line_color="violet", line_dash="solid", line_width = 1)

show(p)

E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: dark blue [renderer: GlyphRenderer(id='579b2c1b-f8ac-4bfc-8941-0f8abd30b56a', ...)]
E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: gree [renderer: GlyphRenderer(id='091523b4-fd04-4714-b9f8-18a03016bae2', ...)]
E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: darkpink [renderer: GlyphRenderer(id='2098112d-5fa8-40c3-8986-20de664db250', ...)]


In [303]:
# PC1 vs PC2

output_file("pc1vs2.html")

p = figure(title="PC1 vs PC2",
           plot_width=1300, plot_height=700,
           x_axis_label='Principal Component 1',
          y_axis_label='Principal Component 2')

pc_x = pcaft_df['PC1']
pc_y = pcaft_df['PC2']

pc_xv = 100 * pcacom_df['PC1']
pc_yv = 100 * pcacom_df['PC2']


p.circle(pc_x, pc_y, color=None, fill_color="red", size=5, fill_alpha = 0.5)
p.segment(0,0, pc_xv, pc_yv, line_color="blue", line_width = 1, line_alpha = 0.1)
p.triangle(pc_xv, pc_yv, color = None, size=5, fill_color="blue", fill_alpha = 0.1, angle=np.arctan(pc_xv/pc_yv))

#p.add_layout(Arrow(end=OpenHead(line_color="blue", line_width=1), x_start=0, y_start=0, x_end=5, y_end=3))

show(p)

E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: dark blue [renderer: GlyphRenderer(id='579b2c1b-f8ac-4bfc-8941-0f8abd30b56a', ...)]
E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: gree [renderer: GlyphRenderer(id='091523b4-fd04-4714-b9f8-18a03016bae2', ...)]
E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: darkpink [renderer: GlyphRenderer(id='2098112d-5fa8-40c3-8986-20de664db250', ...)]
